<a href="https://colab.research.google.com/github/nantmoe-theingi/airbnb-nz-deception-sentiment/blob/main/notebooks/06_run_airbnb_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Create a new .py file and open it for editing
file_name = "run_airbnb_inference.py"
with open(file_name, "w") as f:
    f.write("")  # empty file created
print(f"{file_name} created.")

run_airbnb_inference.py created.


In [ ]:
!python run_airbnb_inference.py \
  --input_parquet data/airbnb_reviews_cleaned.parquet \
  --output_path outputs/airbnb_predictions.parquet \
  --deception_model_dir artifacts/distilbert_deception_sweep/best_model \
  --sentiment_model_dir artifacts/distilbert_sentiment_sweep/best_model

2025-10-30 03:47:43.374668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761796063.394799    2321 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761796063.401123    2321 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761796063.417017    2321 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761796063.417057    2321 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761796063.417068    2321 computation_placer.cc:177] computation placer alr

In [ ]:
!python run_airbnb_inference.py \
  --input_parquet data/airbnb_reviews_cleaned.parquet \
  --output_path outputs/airbnb_predictions_partition/ \
  --partition_by year month \
  --deception_model_dir artifacts/distilbert_deception_sweep/best_model \
  --sentiment_model_dir artifacts/distilbert_sentiment_sweep/best_model

2025-10-28 23:18:29.928352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761693509.948310   16327 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761693509.954317   16327 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761693509.970534   16327 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761693509.970559   16327 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761693509.970563   16327 computation_placer.cc:177] computation placer alr

In [ ]:
import pandas as pd

df = pd.read_parquet("outputs/airbnb_predictions.parquet")
print(df.shape)
print(df.columns.tolist()[-12:])   # show prediction columns
print(df[["sent_pred", "deception_pred"]].value_counts().head())

(1533422, 27)
['deception_logit_0', 'deception_logit_1', 'sent_logit_0', 'sent_logit_1', 'sent_logit_2', 'deception_prob_label_0', 'deception_prob_label_1', 'sent_prob_negative', 'sent_prob_neutral', 'sent_prob_positive', 'deception_pred', 'sent_pred']
sent_pred  deception_pred
2          0                 1197587
           1                  269337
1          0                   43173
0          0                   22383
1          1                     787
Name: count, dtype: int64


In [ ]:
print(df.columns.tolist())

['review_id', 'listing_id', 'date', 'text', 'price_num', 'property_type', 'room_type', 'host_is_superhost', 'neighbourhood', 'region_name', 'region_parent_name', 'region_parent_parent_name', 'region', 'year', 'month', 'deception_logit_0', 'deception_logit_1', 'sent_logit_0', 'sent_logit_1', 'sent_logit_2', 'deception_prob_label_0', 'deception_prob_label_1', 'sent_prob_negative', 'sent_prob_neutral', 'sent_prob_positive', 'deception_pred', 'sent_pred']


In [ ]:
import pandas as pd
import numpy as np

# Load the datasets
reviews = pd.read_csv("data/airbnb_nz_reviews.csv")
listings = pd.read_csv("data/airbnb_nz_listings.csv")
preds = pd.read_parquet("outputs/airbnb_predictions.parquet")

RUNTIME_SECONDS = 3491.4

# Canonicalise keys/dates exactly for the columns you specified ---
def canon_id(s: pd.Series) -> pd.Series:
    return s.astype("string").str.strip().str.replace(r"\.0$", "", regex=True)

for df, cols in [
    (reviews,  ["id", "reviewer_id"]),
    (preds,    ["review_id", "listing_id", "date"]),
    (listings, ["id"]),
]:
    for c in cols:
        if c in df.columns:
            if c == "date":
                df[c] = pd.to_datetime(df[c], errors="coerce")
            else:
                df[c] = canon_id(df[c])

# Use PREDICTIONS as the anchor
# Join preds -> reviews ONLY to fetch reviewer_id for predicted rows
preds_with_reviewer = preds.merge(
    reviews[["id", "reviewer_id"]],
    how="left",
    left_on="review_id",
    right_on="id"
)

# quick sanity checks
predictions_generated = len(preds)  # count of rows that actually have predictions
missing_reviewer_ids = preds_with_reviewer["reviewer_id"].isna().sum()



# Compute metrics restricted to the prediction subset
total_reviews = int("1533422")  # full cleaned corpus (for coverage denominator)
coverage = predictions_generated / total_reviews if total_reviews else np.nan

# Unique listings seen in predictions
unique_listings_pred = preds["listing_id"].nunique()

# Unique reviewers *whose reviews appear in preds*
unique_reviewers_pred = preds_with_reviewer["reviewer_id"].nunique()

# Date span derived from preds['date'] (NOT from all reviews)
if "date" in preds.columns and preds["date"].notna().any():
    date_lo, date_hi = preds["date"].min(), preds["date"].max()
    date_span_txt = f"{date_lo:%b~%Y}--{date_hi:%b~%Y}"
else:
    date_span_txt = "N/A"

throughput = predictions_generated / RUNTIME_SECONDS if RUNTIME_SECONDS else np.nan

# Build summary (values reflect prediction-anchored counts where applicable)
def fmt_int(n): return f"{int(n):,}".replace(",", "{,}")
def fmt_pct(x): return f"{x*100:.2f}\\%"

summary_rows = [
    ("Total reviews processed", "Reviews in the cleaned Airbnb corpus", fmt_int(total_reviews)),
    ("Predictions generated",   "Reviews with valid DistilBERT outputs", fmt_int(predictions_generated)),
    ("Prediction coverage",     "Share of total reviews with predictions", fmt_pct(coverage) if pd.notna(coverage) else "N/A"),
    ("Date span",               "Review period covered by predictions", date_span_txt),
    ("Unique listings",         "Distinct listings represented in predictions", fmt_int(unique_listings_pred)),
    ("Unique reviewers",        "Distinct reviewers represented in predictions", fmt_int(unique_reviewers_pred)),
    ("Runtime (s)",             "Total inference execution time", f"{RUNTIME_SECONDS:,.1f}".replace(",", "{,}")),
    ("Processing throughput",   "Reviews processed per second", f"{throughput:.0f}" if pd.notna(throughput) else "(insert value)"),
]

df_summary = pd.DataFrame(summary_rows, columns=["Metric", "Description", "Value"])
print(df_summary)

# Save CSV
csv_path = "figures/airbnb_inference_integrity_summary.csv"
df_summary.to_csv(csv_path, index=False)
print(f"Saved: {csv_path}")
print(f"Missing reviewer_id after join: {missing_reviewer_ids:,}")


                    Metric                                    Description  \
0  Total reviews processed           Reviews in the cleaned Airbnb corpus   
1    Predictions generated          Reviews with valid DistilBERT outputs   
2      Prediction coverage        Share of total reviews with predictions   
3                Date span           Review period covered by predictions   
4          Unique listings   Distinct listings represented in predictions   
5         Unique reviewers  Distinct reviewers represented in predictions   
6              Runtime (s)                 Total inference execution time   
7    Processing throughput                   Reviews processed per second   

                Value  
0       1{,}533{,}422  
1       1{,}533{,}422  
2            100.00\%  
3  Oct~2022--Aug~2025  
4            39{,}017  
5           758{,}710  
6           3{,}491.4  
7                 439  
Saved: figures/airbnb_inference_integrity_summary.csv
Missing reviewer_id after join: 0
